# Analysis of time series and integration

In [1]:
import copy
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx
from ItalySetup import ItalySetup
from covidOCP import COVIDVaccinationOCP, COVIDParametersOCP
from main import pick_scenario
import seaborn as sns
import datetime
import pickle

nx = 9
states_names = ['S', 'E', 'P', 'I', 'A', 'Q', 'H', 'R', 'V']
outdir = 'model_output/'
when = 'future'

n_int_steps = 5
nnodes = 107  # nodes
ndays = 90

setup = ItalySetup(nnodes, ndays, when)
M = setup.nnodes
N = len(setup.model_days) - 1

with open(f'model_output/parameters_{nnodes}_{when}.pkl', 'rb') as inp:
    p = pickle.load(inp)

Loaded Italy Setup with 107 nodes.


(107, 3)

In [59]:
geodata = pd.read_csv('italy-data/geodata.csv')

geodata = geodata.reset_index(drop=True)

In [69]:
geodata['population'] / agestrat.groupby(['Codice provincia', 'Provincia']).sum().reset_index()['Totale'] * 100

0       99.576047
1       99.189238
2       99.844424
3       99.752106
4       99.423765
          ...    
102     99.491703
103    100.257199
104     99.691404
105     99.689692
106     99.121047
Length: 107, dtype: float64

In [296]:
import casadi as ca
import casadi.tools as cat
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from timeit import default_timer as timer

#if "Agg" not in mpl.get_backend():
#    mpl.interactive(True)
#plt.ion()

nx = 9
states_names = ['S', 'E', 'P', 'I', 'A', 'Q', 'H', 'R', 'V']
mob_scaling = 1e7
nc = 3 # Number of age classes
M = 10
N = 10
nc = 3  # Number of age classes
ages_names = ['Y', 'M', 'O']

In [338]:
dt = (N + 1) / N / n_int_steps

states = cat.struct_symSX(states_names)
[S, E, P, I, A, Q, H, R, V] = states[...]
controls = cat.struct_symSX(['v', 'mob'])
[v, mob] = controls[...]

In [339]:
states = cat.struct_symSX([cat.entry("AG", struct=states, repeat=[nc])])

In [340]:
states = cat.struct_symSX([
    cat.entry("Y", struct=states),
    cat.entry("M", struct=states),
    cat.entry("O", struct=states)])

In [254]:
states

In [341]:
Vars = cat.struct_symMX([
            (
                cat.entry("x", struct=states, repeat=[M, N + 1]),
                cat.entry("u", struct=controls, repeat=[M, N, nc]),
            ),
        ])

In [286]:
print(Vars)

MX.sym with following structure:
Structure with total size 3570.
Structure holding 2 entries.
  Order: [('x', 'u')]
  x = repeated([10, 11]): {Y: {S: 1x1,E: 1x1,P: 1x1,I: 1x1,A: 1x1,Q: 1x1,H: 1x1,R: 1x1,V: 1x1},M: {S: 1x1,E: 1x1,P: 1x1,I: 1x1,A: 1x1,Q: 1x1,H: 1x1,R: 1x1,V: 1x1},O: {S: 1x1,E: 1x1,P: 1x1,I: 1x1,A: 1x1,Q: 1x1,H: 1x1,R: 1x1,V: 1x1}}
  u = repeated([10, 10, 3]): {v: 1x1,mob: 1x1}



In [337]:
Vars['x', 0, 0]

MX(vertsplit(V){0})

In [289]:
print(Vars['x', 0, 0, 'Y' , 'S'])

vertsplit(vertsplit(vertsplit(V){0}){0}){0}


In [290]:
a, b = [0,1], [1,0]

In [294]:
ca.veccat(*Vars['x', :, 3, 'Y', 'E'])

MX(@1=vertsplit(V), vertcat(vertsplit(vertsplit(@1{3}){0}){1}, vertsplit(vertsplit(@1{44}){0}){1}, vertsplit(vertsplit(@1{85}){0}){1}, vertsplit(vertsplit(@1{126}){0}){1}, vertsplit(vertsplit(@1{167}){0}){1}, vertsplit(vertsplit(@1{208}){0}){1}, vertsplit(vertsplit(@1{249}){0}){1}, vertsplit(vertsplit(@1{290}){0}){1}, vertsplit(vertsplit(@1{331}){0}){1}, vertsplit(vertsplit(@1{372}){0}){1}))

In [260]:
print(*Vars['x', 0, 0, 'Y'].cat)

AttributeError: 'MX' object has no attribute 'cat'

In [267]:
print(states)

symbolic SX with following structure:
Structure with total size 27.
Structure holding 3 entries.
  Order: ['Y', 'M', 'O']
  Y = {S: 1x1,E: 1x1,P: 1x1,I: 1x1,A: 1x1,Q: 1x1,H: 1x1,R: 1x1,V: 1x1}
  M = {S: 1x1,E: 1x1,P: 1x1,I: 1x1,A: 1x1,Q: 1x1,H: 1x1,R: 1x1,V: 1x1}
  O = {S: 1x1,E: 1x1,P: 1x1,I: 1x1,A: 1x1,Q: 1x1,H: 1x1,R: 1x1,V: 1x1}



SX([Y_S, Y_E, Y_P, Y_I, Y_A, Y_Q, Y_H, Y_R, Y_V, M_S, M_E, M_P, M_I, M_A, M_Q, M_H, M_R, M_V, O_S, O_E, O_P, O_I, O_A, O_Q, O_H, O_R, O_V])

In [238]:
ca.veccat(*states[...])

NotImplementedError: Wrong number or type of arguments for overloaded function '_veccat'.
  Possible C/C++ prototypes are:
    casadi::casadi_veccat(std::vector< casadi::Sparsity,std::allocator< casadi::Sparsity > > const &)
    casadi::casadi_veccat(std::vector< casadi::MX,std::allocator< casadi::MX > > const &)
    casadi::casadi_veccat(std::vector< casadi::Matrix< casadi_int >,std::allocator< casadi::Matrix< casadi_int > > > const &)
    casadi::casadi_veccat(std::vector< casadi::Matrix< double >,std::allocator< casadi::Matrix< double > > > const &)
    casadi::casadi_veccat(std::vector< casadi::Matrix< casadi::SXElem >,std::allocator< casadi::Matrix< casadi::SXElem > > > const &)


In [282]:
states['Y','S']

SX(Y_S)

In [252]:
print(states['AG'][0])

[AG_0_S, AG_0_E, AG_0_P, AG_0_I, AG_0_A, AG_0_Q, AG_0_H, AG_0_R, AG_0_V]


In [283]:
states.cat

SX([Y_S, Y_E, Y_P, Y_I, Y_A, Y_Q, Y_H, Y_R, Y_V, M_S, M_E, M_P, M_I, M_A, M_Q, M_H, M_R, M_V, O_S, O_E, O_P, O_I, O_A, O_Q, O_H, O_R, O_V])

In [298]:
k = 0
Sk, Ek, Pk, Rk, Ak, Ik, Vk = [ca.veccat(*Vars['x', :, k, ag, 'S']) for ag in ages_names], \
            [ca.veccat(*Vars['x', :, k, ag, 'E']) for ag in ages_names], \
            [ca.veccat(*Vars['x', :, k, ag, 'P']) for ag in ages_names], \
            [ca.veccat(*Vars['x', :, k, ag, 'R']) for ag in ages_names], \
            [ca.veccat(*Vars['x', :, k, ag, 'A']) for ag in ages_names], \
            [ca.veccat(*Vars['x', :, k, ag, 'I']) for ag in ages_names], \
            [ca.veccat(*Vars['x', :, k, ag, 'V']) for ag in ages_names]

In [303]:
np.sum(Sk)

MX(@1=vertsplit(V), @2=vertsplit(@1{0}), @3=vertsplit(@1{41}), @4=vertsplit(@1{82}), @5=vertsplit(@1{123}), @6=vertsplit(@1{164}), @7=vertsplit(@1{205}), @8=vertsplit(@1{246}), @9=vertsplit(@1{287}), @10=vertsplit(@1{328}), @11=vertsplit(@1{369}), ((vertcat(vertsplit(@2{0}){0}, vertsplit(@3{0}){0}, vertsplit(@4{0}){0}, vertsplit(@5{0}){0}, vertsplit(@6{0}){0}, vertsplit(@7{0}){0}, vertsplit(@8{0}){0}, vertsplit(@9{0}){0}, vertsplit(@10{0}){0}, vertsplit(@11{0}){0})+vertcat(vertsplit(@2{1}){0}, vertsplit(@3{1}){0}, vertsplit(@4{1}){0}, vertsplit(@5{1}){0}, vertsplit(@6{1}){0}, vertsplit(@7{1}){0}, vertsplit(@8{1}){0}, vertsplit(@9{1}){0}, vertsplit(@10{1}){0}, vertsplit(@11{1}){0}))+vertcat(vertsplit(@2{2}){0}, vertsplit(@3{2}){0}, vertsplit(@4{2}){0}, vertsplit(@5{2}){0}, vertsplit(@6{2}){0}, vertsplit(@7{2}){0}, vertsplit(@8{2}){0}, vertsplit(@9{2}){0}, vertsplit(@10{2}){0}, vertsplit(@11{2}){0})))

In [344]:
a = Vars['x', 0, 0]

In [357]:
S ,  E  = [], []

In [358]:
S.append(10)

In [360]:
[0, 1]+ [2,3]

[0, 1, 2, 3]